In [72]:
import sys,os
sys.path.append("../")

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import HTML
from sklearn.metrics import pairwise_distances  


%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import entropy
import itertools

import seaborn as sns


from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import pairwise_distances  
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import mutual_info_score
from sklearn.metrics import normalized_mutual_info_score

### Function to take weighted entropy

In [2]:
def weighted_entropy(dataframe, entropy_column, group_columns="cluster", ):
    group_size = dataframe.groupby(group_columns).size()
    group_entropy = dataframe.groupby(group_columns)[entropy_column].apply(lambda x: entropy(x.value_counts().values))
    weighted_entropy = (group_size * group_entropy).mean()
    return weighted_entropy

## reading the change object.

In [3]:
article_name = "John_Logie_Baird"
change_object_dir =  "../data/change objects/"
content_dir = "../data/content/"
filename =  f"{article_name}_change.h5"
change_object_file = os.path.join(change_object_dir, filename)
filename = article_name + ".h5"
filepath = os.path.join(content_dir, filename)
with pd.HDFStore(filepath, 'r') as store:
    token_string_df = store.get("all_tokens")

token_string_df = token_string_df.set_index("token_id")["str"]
token_string_df[-1] = "St@rt"
token_string_df[-2] = "$nd"


if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
change_object_dataframe.shape


change_object_dataframe["ins_length"] = change_object_dataframe["ins_tokens"].str.len()
change_object_dataframe["del_length"] = change_object_dataframe["del_tokens"].str.len()

change_object_dataframe["del_string_tokens"] = change_object_dataframe["del_tokens"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist()))

change_object_dataframe["ins_string_tokens"] = change_object_dataframe["ins_tokens"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist()))
change_object_dataframe["edit_string_tokens"] = change_object_dataframe["ins_string_tokens"] + change_object_dataframe["del_string_tokens"]



change_object_dataframe["left_context"] = change_object_dataframe["left_token"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist())).str.join(" ")


change_object_dataframe["right_context"] = change_object_dataframe["right_token"].apply(
    lambda x:  tuple(token_string_df[np.array(x)].tolist())).str.join(" ")

change_object_dataframe["bykau_cluster"] = pd.Series(-99,index=change_object_dataframe.index)

# change_object_dataframe = change_object_dataframe[["left_context","del_string_tokens","ins_string_tokens", "right_context", 
#                                                    "ins_length", "del_length", "bykau_cluster" ]]
change_object_dataframe.head()

ins_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           10.0   
                                                                             1           32.0   
                                                                             2           34.0   
                                                                             3           50.0   
                                                                             4           57.0   

                                                                                ins_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0         30.0   
                                                                             1         32.0   
                                                                             2         34.0   
                                                                             3         55.0   
                                                                             4         73.0   

                                                                                left_neigh  \
from revision id to revision id timestamp           timegap           editor                 
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           9   
                                                                             1          10   
                                                                             2          11   
                                                                             3          26   
                                                                             4          32   

                                                                                right_neigh  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           10   
                                                                             1           11   
                                                                             2           12   
                                                                             3           32   
                                                                             4           36   

                                                                                del_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0           -1.0   
                                                                             1           -1.0   
                                                                             2           -1.0   
                                                                             3           27.0   
                                                                             4           33.0   

                                                                                del_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
203693           203699         2002-09-08 14:05:32 194 days 22:14:17 3646   0         -1.0   
                                                                             1         -1.0   
                                                                             2         -1.0   
                                                                             3         31.0   
                                                                             4         35.0   

                                                               

## Reduce the change object.

In [4]:
ins_and_del = change_object_dataframe[(change_object_dataframe["ins_string_tokens"]!=()) & (change_object_dataframe["del_string_tokens"]!=())]
display(ins_and_del.shape)

(2018, 20)

### Removing change object with insert or delete token size more than five.

In [5]:
reduced_ins_and_del = ins_and_del[~((ins_and_del["ins_length"] >5 ) | (ins_and_del["del_length"] >5) )]
reduced_ins_and_del.shape

(1542, 20)

In [6]:
reduced_ins_and_del.reset_index(drop=True)[["ins_string_tokens", "del_string_tokens"]].sample(3)

,ins_string_tokens,del_string_tokens
11,"(2004,)","(1929,)"
906,"(2010,)","(2012,)"
511,"([[, lomond, school, ]])","(lomond, school)"


### Removing low user support tokens

In [7]:
bykau_change_object = reduced_ins_and_del.groupby("ins_string_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object = bykau_change_object.groupby("del_string_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object.shape

(493, 20)

#### read annotations

In [8]:
file_name = article_name + "_FULL.csv"
annotation_dir = "../data/annotation/"
full_file_path = os.path.join(annotation_dir, file_name)
annotation_df = pd.read_csv(full_file_path)
annotation_df = annotation_df[["revid_ctxt", "token_id",
                               "rev_id", "nationality", "birth_place" ]]

In [9]:
# file_name = article_name + "_FULL.csv"
# annotation_dir = "../data/annotation/"
# full_file_path = os.path.join(annotation_dir, file_name)
# annotation_df = pd.read_csv(full_file_path)

## Function for weighted entropy

In [44]:
def weighted_token_entropy(dataframe, group_by):
    cluster_sizes = dataframe.groupby(group_by).size()
    token_entropy_clusters = dataframe.groupby(group_by)["edit_string_tokens"].apply(
                    lambda token_tuples: entropy(pd.Series(
                    [token for token_tuple in token_tuples.tolist() for token in token_tuple]
                    ).value_counts().values))
    cluster_entropy = (cluster_sizes * token_entropy_clusters).sum()
    return cluster_entropy

### Clustering using jaccard

##### Defining jaccard similarity function.

In [10]:
def bykau_distances(dataframe, context = 8):
    left_neighbours = dataframe["left_context"].apply(lambda x: x.split(" ")[-context:])
    right_neighbours = dataframe["right_context"].apply(lambda x: x.split(" ")[:context])
    
    neighbour_tokens = left_neighbours + right_neighbours
    neighbour_tokens_set = neighbour_tokens.apply(lambda x: np.unique(x))
    
    neighbour_vec = MultiLabelBinarizer().fit_transform(neighbour_tokens_set)
    return pairwise_distances(neighbour_vec, metric="jaccard")
#     db = DBSCAN(eps=eps, min_samples=min_samples, metric='jaccard').fit(neighbour_vec)
#     return db.labels_ 

In [11]:
def bykau_cluster(distances, eps=0.75, min_samples=5):
    db = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(distances)
    return db.labels_ 


In [12]:
# %%time
# r_threshold = 8
# cutoff_threshold = 0.75
# # edit_tokens = change_object_dataframe["ins_tokens"] + change_object_dataframe["del_tokens"]
# left_neighbours = change_object_dataframe["left_context"].apply(lambda x: x.split(" ")[-r_threshold:])
# right_neighbours = change_object_dataframe["right_context"].apply(lambda x: x.split(" ")[:r_threshold])
# neighbour_tokens = left_neighbours + right_neighbours

# # bykau_change_object["edit_tokens"] = edit_tokens.apply(lambda x: np.unique(x))
# neighbour_tokens_set = neighbour_tokens.apply(lambda x: np.unique(x))
# neighbour_vec = MultiLabelBinarizer().fit_transform(neighbour_tokens_set)

# db = DBSCAN(eps=cutoff_threshold, min_samples=5, metric='jaccard').fit(neighbour_vec)
# # change_object_dataframe.loc[change_object_dataframe.index,"bykau_cluster"] = db.labels_

#### Saving change object and its clusters

In [13]:
# bykau_dir =  "../data/bykau_change_object/"
# filename =  f"{article_name}_without_optimization.h5"

# change_object_file = os.path.join(bykau_dir, filename)
# with pd.HDFStore(change_object_file, 'w') as store:
#     store.put("data", change_object_dataframe["bykau_cluster"],)

### Uncomment for seeing the content of each cluster in an ipywidget User interface

In [14]:
# repers_first =  change_object_dataframe.groupby("bykau_cluster")[["left_context","del_string_tokens", "ins_string_tokens", "right_context"]].apply(lambda x: x.style.render())
# @interact( clusters_html=fixed(repers_first), group=change_object_dataframe["bykau_cluster"].unique().tolist())
# def display_clusters(clusters_html, group):
#      return display(HTML(clusters_html.loc[group]))

#### Making annotation file with cluster

In [15]:
# def token_in_gap(ann, gap):
#     context_gap = gap.loc[ann[['revid_ctxt', 'rev_id']]]
#     clusters = context_gap.loc[ context_gap["token_id"].apply(
#             lambda x: ann["token_id"] in x), ["bykau_cluster"]].values
#     if clusters.size >0:
#             clusters = pd.Series(clusters[0],index=["bykau_cluster",])
#     else:
#         clusters = pd.Series([-10], index=["bykau_cluster",])
#     return clusters

In [16]:
def token_in_gap(ann, gap_df, gap_cluster_df):
    context_gap = gap_df.loc[ann[['revid_ctxt', 'rev_id']]]
    context_cluster = gap_cluster_df.loc[ann[['revid_ctxt', 'rev_id']]]
    clusters = context_cluster.loc[ context_gap["token_id"].apply(
            lambda x: ann["token_id"] in x),:].values
    if clusters.size >0:
            clusters = pd.Series(clusters[0],index=gap_cluster_df.columns)
    else:
        clusters = pd.Series(-10, index=gap_cluster_df.columns)
    return clusters

In [17]:
def evaluate_bykau(change_object_dataframe, annotation_df):
    ins_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id', "bykau_cluster"]].values

    # delete array is always done in from revision so taking it and leaving other change object where delete does not come.
    del_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id',"bykau_cluster"]].values

    gap_array = np.concatenate([ins_array,del_array], axis=0)
    gap_df = pd.DataFrame(gap_array,columns= ["revid_ctxt", "token_id", "rev_id","bykau_cluster"])
    gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])
    
    annotation_df["bykau_cluster"] = annotation_df.apply(token_in_gap, axis=1, args=(gap_df,))
    nationality_cluster = np.zeros((annotation_df.shape[0]))
    nationality_cluster[annotation_df["nationality"].str.strip() == "Y"] = 1
    annotation_df["nationality_cluster"] = nationality_cluster

    evaluation_score = pd.Series(index=["rand", "entropy",])
    evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], nationality_cluster)
    evaluation_score["entropy"] = weighted_entropy(annotation_df, group_columns="bykau_cluster", entropy_column="nationality_cluster")
    return evaluation_score   

In [18]:
context_array  = np.array([2, 4, 8, 15, 30])
eps_array =[0.001, 0.025, 0.05, 0.1, 0.2, 0.4, 0.8]

min_samples_array = 2 
all_combinations = list(itertools.product(context_array, eps_array))

idx = pd.MultiIndex.from_product([context_array, eps_array],
                                names=["context","eps"])


all_combinations = list(itertools.product(context_array, eps_array))


bykau_cluster_df = pd.DataFrame(columns=idx)

bykau_evaluation_df = pd.DataFrame(index=idx, columns=["rand", "entropy"])


In [19]:
for context in context_array:
    distances = bykau_distances(bykau_change_object, context = context)
    for eps in eps_array:
        clusters = bykau_cluster(distances, eps=eps, min_samples=2)
        
        bykau_cluster_df[context,eps] = pd.Series(clusters)

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec

In [20]:
# insert array is always done in to revision so taking it and leaving other change object where 
ins_array = bykau_change_object.reset_index().loc[
    bykau_change_object["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id']].values
ins_cluster = bykau_cluster_df.loc[
    bykau_change_object["ins_start_pos"].values != -1, :]

# delete array is always done in from revision so taking it and leaving other change object where delete does not come.
del_array = bykau_change_object.reset_index().loc[
    bykau_change_object["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id']].values
del_cluster = bykau_cluster_df.loc[
    bykau_change_object["del_start_pos"].values != -1, :]

gap_array = np.concatenate([ins_array,del_array], axis=0)
gap_df = pd.DataFrame(gap_array,columns=["revid_ctxt", "token_id",
                               "rev_id"])

gap_cluster= pd.concat([ins_cluster, del_cluster], axis=0)
gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])
gap_cluster_df = pd.concat([ins_cluster, del_cluster], axis=0)

gap_cluster_df.index=gap_df.index

# Finding the tokens who were in the gap.
al_combination_clusters_df = annotation_df.apply(token_in_gap, axis=1, args=(gap_df, gap_cluster_df))

annotation_clusters = pd.concat([annotation_df, al_combination_clusters_df], axis=1)



true_labels = np.zeros((annotation_df.shape[0]))
true_labels[(annotation_df["nationality"].str.strip() == "Y").values] = 1
annotation_df["nationality"] = true_labels
#true_labels[true_lable_df["birth_place"].str.strip() == "Y"] = 2



In [21]:
for context, eps in all_combinations:
    bykau_evaluation_df.loc[(context, eps),"entropy"] = weighted_entropy(annotation_clusters, 
                                                                                entropy_column="nationality", 
                                                                                group_columns=(context, eps))
    bykau_evaluation_df.loc[(context, eps),"rand"] = adjusted_rand_score(annotation_clusters[(context, 
                                                                                                     eps)], 
                                                                                true_labels)

In [22]:
bykau_evaluation_df["entropy"].sort_values().iloc[0:50]

context  eps  
15       0.001    16.6146
         0.025    16.6146
         0.100    16.9121
30       0.001    17.0722
         0.025    17.4926
         0.050    17.8787
8        0.050    18.3545
         0.001    18.3545
         0.025    18.3545
15       0.050    18.5134
8        0.100    18.7685
4        0.050    18.7923
         0.001    18.7923
         0.100    18.7923
         0.025    18.7923
         0.200    19.9241
30       0.100    20.3486
2        0.001    21.0485
         0.025    21.0485
         0.050    21.0485
         0.100    21.0485
         0.200    21.0485
8        0.200    21.0932
15       0.200    21.6851
30       0.200    24.4529
15       0.400    34.5426
8        0.400    36.4751
30       0.400    37.1242
4        0.400    41.6736
2        0.400    54.8317
15       0.800    181.064
2        0.800    183.683
4        0.800    184.524
8        0.800    370.204
30       0.800    370.204
Name: entropy, dtype: object

In [23]:
context_array  = np.array([2, 4, 8, 15, 30])
eps_array =[0.001, 0.025, 0.05, 0.1, 0.2, 0.4, 0.8]

min_samples_array = [2] 
# all_combinations = list(itertools.product(context_array, eps_array))
all_combinations_without_optimization = list(itertools.product(context_array, eps_array))

idx_without_optimization  = pd.MultiIndex.from_product([context_array, eps_array],
                                names=["context","eps"])

# idx_without_optimization = pd.MultiIndex.from_product([context_array, eps_array, min_samples_array],
#                                 names=["context","eps", "min_samples"])
bykau_evaluation_without_optimization = pd.DataFrame(index=idx_without_optimization, 
                                                     columns=["rand", "entropy"])



bykau_without_optimization_cluster_df = pd.DataFrame(columns=idx_without_optimization)



In [24]:
%%time
for context in context_array:
    distances = bykau_distances(change_object_dataframe, context = context)
    for eps in eps_array:
        clusters = bykau_cluster(distances, eps=eps, min_samples=2)
        
        bykau_without_optimization_cluster_df[context,eps] = pd.Series(clusters)

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda3/envs/wrod2vec

CPU times: user 2min 10s, sys: 21.6 s, total: 2min 32s
Wall time: 2min 12s


In [25]:
# insert array is always done in to revision so taking it and leaving other change object where 
ins_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["ins_start_pos"].values != -1, 
                  ["to revision id","ins_tokens", 'to revision id']].values
ins_cluster = bykau_without_optimization_cluster_df.loc[
    change_object_dataframe["ins_start_pos"].values != -1, :]

# delete array is always done in from revision so taking it and leaving other change object where delete does not come.
del_array = change_object_dataframe.reset_index().loc[
    change_object_dataframe["del_start_pos"].values != -1, 
                  ["from revision id","del_tokens", 'to revision id']].values
del_cluster = bykau_without_optimization_cluster_df.loc[
    change_object_dataframe["del_start_pos"].values != -1, :]

gap_array = np.concatenate([ins_array,del_array], axis=0)
gap_df = pd.DataFrame(gap_array,columns=["revid_ctxt", "token_id",
                               "rev_id"])

gap_cluster= pd.concat([ins_cluster, del_cluster], axis=0)
gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])
gap_cluster_df = pd.concat([ins_cluster, del_cluster], axis=0)

gap_cluster_df.index=gap_df.index

# Finding the tokens who were in the gap.
al_combination_clusters_df = annotation_df.apply(token_in_gap, axis=1, args=(gap_df, gap_cluster_df))

annotation_clusters = pd.concat([annotation_df, al_combination_clusters_df], axis=1)



# true_labels = np.zeros((annotation_df.shape[0]))
# true_labels[(annotation_df["nationality"].str.strip() == "Y").values] = 1
# annotation_df["nationality"] = true_labels
#true_labels[true_lable_df["birth_place"].str.strip() == "Y"] = 2



In [26]:
for context, eps in all_combinations_without_optimization:
    bykau_evaluation_without_optimization.loc[(context, eps),"entropy"] = weighted_entropy(annotation_clusters, 
                                                                                entropy_column="nationality", 
                                                                                group_columns=(context, eps))
    bykau_evaluation_without_optimization.loc[(context, eps),"rand"] = adjusted_rand_score(annotation_clusters[(context, 
                                                                                                     eps)], 
                                                                                true_labels)

In [27]:
# for context, eps, min_samples in all_combinations_without_optimization:
#     print(f"processing eps {eps}, min samples {min_samples} and context {context}")
#     change_object_dataframe.loc[change_object_dataframe.index,
#                             "bykau_cluster"] = bykau_cluster(change_object_dataframe, 
#                                                              context=int(context), eps=eps, min_samples=min_samples)
#     bykau_evaluation_without_optimization.loc[context, eps, 
#                             min_samples] =evaluate_bykau(change_object_dataframe, annotation_df)


In [28]:
#bykau_evaluation_without_optimization["entropy"].sort_values().iloc[0:200]

bykau_evaluation_without_optimization.reset_index().set_index(["eps", "context"])["entropy"].sort_values().iloc[0:50]

eps    context
0.100  30         1.70298
0.200  15         1.76482
0.100  15         1.87732
0.001  2          1.90188
0.025  2          1.90188
0.050  2          1.90188
0.100  2          1.90188
0.200  2          1.90188
       30         1.92995
       8          1.95816
0.050  30         2.07604
0.001  4          2.11921
0.025  4          2.11921
0.050  4          2.11921
0.100  4          2.11921
0.200  4          2.17222
0.100  8          2.48144
0.050  15         2.51455
0.025  30         2.57668
       8           2.6191
0.001  8           2.6191
0.050  8           2.6191
0.001  30         2.65478
       15         2.75388
0.025  15         2.75388
0.400  8          4.49188
       4          5.47147
       15         5.85951
       30         9.00075
       2          9.07192
0.800  2          262.476
       8          350.297
       4          350.394
       15         525.609
       30         525.609
Name: entropy, dtype: object

In [55]:
# # insert array is always done in to revision so taking it and leaving other change object where 
# ins_array = change_object_dataframe.reset_index().loc[
#     change_object_dataframe["ins_start_pos"].values != -1, 
#                   ["to revision id","ins_tokens", 'to revision id']].values

# # delete array is always done in from revision so taking it and leaving other change object where delete does not come.
# del_array = change_object_dataframe.reset_index().loc[
#     change_object_dataframe["del_start_pos"].values != -1, 
#                   ["from revision id","del_tokens", 'to revision id',"bykau_cluster"]].values

# gap_array = np.concatenate([ins_array,del_array], axis=0)
# gap_df = pd.DataFrame(gap_array,columns= ["revid_ctxt", "token_id", "rev_id","bykau_cluster"])
# gap_df = gap_df.set_index(['revid_ctxt', 'rev_id'])

# Finding the tokens who were in the gap.
# annotation_df["bykau_cluster"] = annotation_df.apply(token_in_gap, axis=1, args=(gap_df,))

# nationality_cluster = np.zeros((annotation_df.shape[0]))
# nationality_cluster[annotation_df["nationality"].str.strip() == "Y"] = 1
# annotation_df["nationality_cluster"] = nationality_cluster

# evaluation_score = pd.Series(index=["rand", "entropy",])
# evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], nationality_cluster)
# evaluation_score["entropy"] = weighted_entropy(annotation_df, group_columns="bykau_cluster", entropy_column="nationality_cluster")
# evaluation_score

# both_cluster = np.zeros((annotation_df.shape[0]))
# both_cluster[annotation_df["nationality"].str.strip() == "Y"] = 1
# both_cluster[annotation_df["birth_place"].str.strip() == "Y"] = 2
# annotation_df["both_cluster"] = both_cluster

# evaluation_score = pd.Series(index=["rand", "entropy", ])
# evaluation_score["rand"] = adjusted_rand_score( annotation_df["bykau_cluster"], both_cluster)

# evaluation_score["entropy"] = weighted_entropy(annotation_df, group_columns="bykau_cluster", entropy_column="both_cluster")

# evaluation_score


In [44]:
def weighted_token_entropy(dataframe, group_by):
    cluster_sizes = dataframe.groupby(group_by).size()
    token_entropy_clusters = dataframe.groupby(group_by)["edit_string_tokens"].apply(
                    lambda token_tuples: entropy(pd.Series(
                    [token for token_tuple in token_tuples.tolist() for token in token_tuple]
                    ).value_counts().values))
    cluster_entropy = (cluster_sizes * token_entropy_clusters).sum()
    return cluster_entropy

In [60]:
bykau_cluster_df.index = bykau_change_object.index

bykau_optimised = pd.concat([change_object_dataframe, bykau_without_optimization_cluster_df], axis=1)

In [61]:
entropy_series_with_optimization = pd.Series(index=all_combinations)

for context_eps_tuple in all_combinations_without_optimization:
    bykau_evaluation_df.loc[context_eps_tuple,
                                              "token_entropy"] = weighted_token_entropy(bykau_optimised, 
                                                                                        context_eps_tuple)
#     entropy_series_with_optimization[context_eps_tuple]= weighted_token_entropy(bykau_optimised, context_eps_tuple)
# all_combinations_without_optimization[0]

In [62]:
# entropy_series_with_optimization.sort_values()

In [63]:
bykau_without_optimization_cluster_df.index = change_object_dataframe.index

without_optimised = pd.concat([change_object_dataframe, bykau_without_optimization_cluster_df], axis=1)

In [64]:
# entropy_series_without_optimization = pd.Series(index=all_combinations_without_optimization)

for context_eps_tuple in all_combinations_without_optimization:
    bykau_evaluation_without_optimization.loc[context_eps_tuple,
                                              "token_entropy"] = weighted_token_entropy(without_optimised, 
                                                                                        context_eps_tuple)

#     entropy_series_without_optimization[context_eps_tuple]= weighted_token_entropy(without_optimised, context_eps_tuple)
# all_combinations_without_optimization[0]

In [65]:
# entropy_series_without_optimization.sort_values()

In [66]:
bykau_evaluation_without_optimization.head()

rand  entropy  token_entropy
context eps                                     
2       0.001  0.0358306  1.90188   17480.153219
        0.025  0.0358306  1.90188   17480.153219
        0.050  0.0358306  1.90188   17480.153219
        0.100  0.0358306  1.90188   17480.153219
        0.200  0.0358306  1.90188   17480.153219

In [68]:
bykau_evaluation_df.head()

rand  entropy  token_entropy
context eps                                    
2       0.001  0.437664  21.0485   17480.153219
        0.025  0.437664  21.0485   17480.153219
        0.050  0.437664  21.0485   17480.153219
        0.100  0.437664  21.0485   17480.153219
        0.200  0.437664  21.0485   17480.153219

In [74]:
bykau_evaluation_df.astype(np.float64).corr()

,rand,entropy,token_entropy
rand,1.000000,0.910926,0.924432
entropy,0.910926,1.000000,0.855127
token_entropy,0.924432,0.855127,1.000000


In [73]:
bykau_evaluation_without_optimization.astype(np.float64).corr()

,rand,entropy,token_entropy
rand,1.000000,-0.380560,-0.647538
entropy,-0.380560,1.000000,0.882049
token_entropy,-0.647538,0.882049,1.000000
